In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

from shutil import copy

from PIL import Image
import imageio

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
colours = {'.':np.array([119,119,119]), '#':np.array([220,  5, 12]), '@':np.array([ 78,178,101]),
           'O':np.array([ 25,101,176]), '[':np.array([ 25,101,176]), ']':np.array([ 65,125,191])}

def parse_data(data, part=1):
    graph = []
    sequence = ''
    seq = False
    for line in data:
        line = line.strip()
        
        if len(line) == 0:
            seq = True
        elif seq:
            sequence += line
        else:
            if part == 1:
                graph.append(list(line))
            if part == 2:
                graph_line = []
                for char in list(line):
                    if char == '#':
                        graph_line.append('#')
                        graph_line.append('#')
                    elif char == '.':
                        graph_line.append('.')
                        graph_line.append('.')
                    elif char == 'O':
                        graph_line.append('[')
                        graph_line.append(']')
                    elif char =='@':
                        graph_line.append('@')
                        graph_line.append('.')
                graph.append(graph_line)
            
    graph = np.array(graph)
    start = np.where(graph == '@')
    graph[start] = '.'
    start = [start[0][0], start[1][0]]
    
    if part == 1:
        image = np.ones((graph.shape[0],graph.shape[1]*2,3), dtype=int)
        for x, line in enumerate(graph):
            for y, char in enumerate(line):
                y += int(graph.shape[1]//2)
                image[x,y] = colours[char]
    if part == 2:
        image = np.ones((graph.shape[0],graph.shape[1],3), dtype=int)*119
        for x, line in enumerate(graph):
            for y, char in enumerate(line):
                image[x,y] = colours[char]
                
            
    return np.array(graph), np.array(start), deque(list(sequence)), image

def print_graph(graph, pos):
    graph[pos[0],pos[1]] = '@'
    for line in graph:
        print(''.join(line))
    graph[pos[0],pos[1]] = '.'
    
def move_logic_part1(graph, pos, move, dirs, nxt_pos, image):
    if graph[nxt_pos[0],nxt_pos[1]] == '.':
            pos = nxt_pos
            
    elif graph[nxt_pos[0],nxt_pos[1]] == 'O':
        nxt_nxt_pos = deepcopy(nxt_pos)

        while True:
            nxt_nxt_pos += dirs[move]

            if graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '#':
                break

            elif graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '.':
                #move everything
                while nxt_nxt_pos[0] != nxt_pos[0] or nxt_nxt_pos[1] != nxt_pos[1]:
                    graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] = 'O'
                    #image[nxt_nxt_pos[0],nxt_nxt_pos[1]+int(graph.shape[1]//2)] = colours['O']
                    nxt_nxt_pos -= dirs[move]

                graph[nxt_pos[0], nxt_pos[1]] = '.'
                #image[nxt_pos[0],nxt_pos[1]+int(graph.shape[1]//2)] = colours['.']
                pos = nxt_pos
                break
    return graph, pos, image

def move_logic_part2(graph, pos, move, dirs, nxt_pos, image):
    if graph[nxt_pos[0],nxt_pos[1]] == '.':
        return graph, nxt_pos, image
            
    elif graph[nxt_pos[0],nxt_pos[1]] == '#':
        return graph, pos, image
            
    elif move == '<' or move == '>':
        nxt_nxt_pos = deepcopy(nxt_pos)
    
        while True:
            nxt_nxt_pos += 2*dirs[move]
    
            if graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '#':
                break
    
            elif graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] == '.':
                #move everything
                while nxt_nxt_pos[0] != nxt_pos[0] or nxt_nxt_pos[1] != nxt_pos[1]:
                    graph[nxt_nxt_pos[0],nxt_nxt_pos[1]] = graph[nxt_nxt_pos[0]-dirs[move][0],
                                                                 nxt_nxt_pos[1]-dirs[move][1]]
                    #image[nxt_nxt_pos[0],nxt_nxt_pos[1]] = colours[graph[nxt_nxt_pos[0]-dirs[move][0],
                    #                                                     nxt_nxt_pos[1]-dirs[move][1]]]
                    nxt_nxt_pos -= dirs[move]
    
                graph[nxt_pos[0], nxt_pos[1]] = '.'
                #image[nxt_pos[0],nxt_pos[1]] = colours['.']
                pos = nxt_pos
                break
                
    elif move == '^' or move == 'v':
        x = nxt_pos[0]
        dx = dirs[move][0]
        ys = {}
        
        nxt_ys = []
        nxt_ys.append(nxt_pos[1])
        if graph[nxt_pos[0],nxt_pos[1]] == '[':
            nxt_ys.append(nxt_pos[1]+1)
        else:
            nxt_ys.append(nxt_pos[1]-1)
        nxt_ys.sort()
        ys[x] = nxt_ys
            
        while True:
            nxt_x = x + dx
            
            if '#' in graph[nxt_x][ys[x]]:
                break
                
            nxt_y = []
            for y in ys[x]:
                if graph[nxt_x][y] == '[':
                    if y not in nxt_y:
                        nxt_y.append(y)
                    if y+1 not in nxt_y:
                        nxt_y.append(y+1)
                        
                elif graph[nxt_x][y] == ']':
                    if y not in nxt_y:
                        nxt_y.append(y)
                    if y-1 not in nxt_y:
                        nxt_y.append(y-1)
                        
            if len(nxt_y) > 0:
                nxt_y.sort()
                ys[nxt_x] = nxt_y
                x = nxt_x
            else:
                #move everything
                xes = list(ys.keys())
                if move == '^':
                    xes.sort()
                else: #move == 'v'
                    xes.sort(reverse=True)
                    
                for x in xes:
                    for y in ys[x]:
                        graph[x+dx][y] = graph[x][y]
                        if x-dx not in xes or y not in ys[x-dx]:
                            graph[x][y] = '.'
                pos = nxt_pos
                break
            
    return graph, pos, image

#def save_frame(image, pos, frame_num, offset=0):
#    image[pos[0],pos[1]+offset] = colours['@']
#    
#    frame = Image.fromarray(image.astype('uint8'), mode='RGB')
#    frame = frame.resize((frame.size[0]*8,frame.size[1]*8), resample=Image.NEAREST)
#    frame.save('./Day15-Frames/day15_'+str(frame_num).zfill(5)+'.png')
#    
#    image[pos[0],pos[1]+offset] = colours['.']
#    return

def save_frame(graph, pos, part, frame_num):
    graph[pos[0],pos[1]] = '@'
    if part == 1:
        image = np.ones((graph.shape[0],graph.shape[1]*2,3), dtype=int)
        for x, line in enumerate(graph):
            for y, char in enumerate(line):
                y += int(graph.shape[1]//2)
                image[x,y] = colours[char]
    if part == 2:
        image = np.ones((graph.shape[0],graph.shape[1],3), dtype=int)*119
        for x, line in enumerate(graph):
            for y, char in enumerate(line):
                image[x,y] = colours[char]
    graph[pos[0],pos[1]] = '.'
    
    frame = Image.fromarray(image.astype('uint8'), mode='RGB')
    frame = frame.resize((frame.size[0]*8,frame.size[1]*8), resample=Image.NEAREST)
    frame.save('./Day15-Frames/day15_'+str(frame_num).zfill(5)+'.png')

    return

def run_robot(graph, pos, sequence, image, part=1, prnt=False):
    dirs = {'^':np.array([-1,0]), 'v':np.array([1,0]), '>':np.array([0,1]), '<':np.array([0,-1])}
    
    if part == 1:
        count = 0
        #save_frame(image, pos, count, int(graph.shape[1]//2))
    else:
        count = part+1
        part = 2
        #save_frame(image, pos, count)
    save_frame(graph, pos, part, count)
    
    if prnt:
        print('Initial state')
        print_graph(graph, pos)
        print()
        plt.imshow(image)
        plt.show()
        return graph, count
    
    while len(sequence):
        count += 1
        move = sequence.popleft()
        nxt_pos = pos + dirs[move]
        
        if part == 1:
            graph, pos, image = move_logic_part1(graph, pos, move, dirs, nxt_pos, image)
            #save_frame(image, pos, count, int(graph.shape[1]//2))
            save_frame(graph, pos, part, count)
        else:
            graph, pos, image = move_logic_part2(graph, pos, move, dirs, nxt_pos, image)
            #save_frame(image, pos, count)
            save_frame(graph, pos, part, count)
            
        if prnt:
            print('Move:', move)
            print_graph(graph, pos)
            print()
                    
    return graph, count

def gps_score(graph):
    boxes = np.where(np.logical_or(graph == 'O', graph == '['))
    score = 0
    
    for i in range(0, len(boxes[0])):
        score += (boxes[0][i]*100) + boxes[1][i]
        
    return score

def run(data, prnt=False):
    graph, start, sequence, image = parse_data(data)
    graph, count = run_robot(graph, start, sequence, image, prnt=prnt)
    score = gps_score(graph)
    print('Part 1 result:', score)
    
    for t in range(count+1, count+121):
        copy('./Day15-Frames/day15_'+str(count).zfill(5)+'.png',
             './Day15-Frames/day15_'+str(t).zfill(5)+'.png')

    graph, start, sequence, image = parse_data(data, part=2)
    graph, count = run_robot(graph, start, sequence, image, part=count+120, prnt=prnt)
    score = gps_score(graph)
    print('Part 2 result:', score)
    
    for t in range(count+1, count+121):
        copy('./Day15-Frames/day15_'+str(count).zfill(5)+'.png',
             './Day15-Frames/day15_'+str(t).zfill(5)+'.png')

In [3]:
with open('input_day15.txt', 'r') as f:
    data = f.readlines()
    f.close()
    
run(data)

Part 1 result: 1478649
Part 2 result: 1495455
